In [1]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlretrieve

chrome_options = Options()
chrome_options.add_argument("--start-maximized")

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=chrome_options)

url = 'https://www.countdowntopregnancy.com/pregnancy-test-gallery/'

positive_dir = 'positive_images'
negative_dir = 'negative_images'
os.makedirs(positive_dir, exist_ok=True)
os.makedirs(negative_dir, exist_ok=True)

def apply_filter(filter_type):
    filter_button = driver.find_element(By.ID, 'filterDropDown')
    filter_button.click()
    time.sleep(1)


    action = ActionChains(driver)
    filter_menu = driver.find_element(By.CLASS_NAME, 'hidden-xs')
    action.move_to_element(filter_menu).perform()
    time.sleep(1)
    
    filter_option = driver.find_element(By.XPATH, f"//a[contains(text(), '{filter_type}')]")
    filter_option.click()
    time.sleep(2)

def download_images(directory, page_count=60):
    for page in range(1, page_count + 1):
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'img-responsive'))
        )
        
        images = driver.find_elements(By.CLASS_NAME, 'img-responsive')
        
        for idx, img in enumerate(images):
            src = img.get_attribute('src')
            if src.startswith('/'):
                src = f"https://www.countdowntopregnancy.com{src}"
            urlretrieve(src, os.path.join(directory, f'{page}_{idx+1}.jpg'))
        
        try:
            next_button = driver.find_element(By.XPATH, "//i[@class='fa fa-fw fa-chevron-right']")
            driver.execute_script("arguments[0].click();", next_button)
        except Exception as e:
            print(f"No more pages after {page}: {e}")
            break
        
        time.sleep(3)

# Start the scraping process
driver.get(url)

# Apply Positive filter and download images
apply_filter('Positive')
download_images(positive_dir)

# Apply Negative filter and download images
apply_filter('Negative')
download_images(negative_dir)

driver.quit()